### Image Morphing

In [116]:
#import necessary libraries

import cv2
import dlib
import numpy as np
from scipy.spatial import Delaunay
import imageio
from google.colab.patches import cv2_imshow
from imutils import face_utils
from PIL import Image

In [117]:
#run once to load the .dat file
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK

!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

datFile =  "/content/shape_predictor_68_face_landmarks.dat"

In [118]:
def cnvt_colour_bgr2rgb(image): 
  return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [119]:
def find_facial_tilepts(img):
    # Detect faces in the image
    faces = detector(img)

    no_pnt=0
    # Get the facial landmarks for the first detected face
    shape = predictor(img, faces[0])

    no_pnt+=1
    # print(no_pnt)
    # Extract the x and y coordinates of each landmark and store them in a numpy array
    tile_pts_ = np.array([[p.x, p.y] for p in shape.parts()], dtype=np.int32)

    # Create a list of numpy arrays containing the landmarks for each face detected in the image
    tile_pts = [tile_pts_]

    # Return the list of landmarks
    return tile_pts


In [120]:
# This function takes an image as input and returns a list of coordinates that mark the facial features

def mark_coord(img):
    triangle=[]
    h, w, c = img.shape
    # Convert the image to grayscale
    img_bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the face and get its coordinates
    rect_cordinates = detector(img_bw, 0)

    # Use the facial landmark predictor to get the coordinates of the facial features
    shaped = face_utils.shape_to_np(predictor(img_bw, rect_cordinates[0]))
    for cd in shaped:
      # Create a list of tuples that contains the (x, y) coordinates of the facial features
      triangle.append((int(cd[0]),int(cd[1])) )
     # Add four additional corner points to the list to define a rectangular region that contains the entire image
    triangle += [(0, 0), (0, h-1), (w-1, h-1), (w-1, 0)]
    return triangle

In [121]:
#delaunay triangulation using cv2.SudDiv2D

def delaunay_triangulation(points, image):
    # Create a Subdiv2D object and insert points into it
    no_pnt=0
    height, width, channels = image.shape
    subdiv = cv2.Subdiv2D((0, 0, width, height))
    tri=[]
    no_pnt+=1
    for point in points:
        subdiv.insert(point)
    for i in range(no_pnt):
        tri.append(i)

    # Retrieve the list of triangles from the Subdiv2D object
    triangle_list = subdiv.getTriangleList()
    
    return triangle_list

In [122]:
def check_index(pt, triangle):

    t = None
    for i, index in enumerate(triangle):

        if pt == index:
            t = i
            break
    return t

In [123]:
def Transform_mid(tri_1, tri_2, rect, rect1, rect2):
    tri_1_cropped, tri_2_cropped, t_cropped = [],[],[]
    # rect = cv2.boundingRect(np.float32([tri_1]))
    # rec_coord_2 = cv2.boundingRect(np.float32([tri_2]))

    size = (rect[2], rect[3])
  
    return tri_1_cropped, tri_2_cropped, t_cropped

In [124]:
#Combine input and output via affine transform
def Affine_transform(img1, img2, img, tri_1, tri_2, interm_pt, alpha):
    rec_coord = cv2.boundingRect(np.float32([interm_pt]))
    t1,t2=[],[]
    rec_cd_1=np.float32([tri_1])
    rec_cd_2= np.float32([tri_2])
    pt=3
    tri_1_cr, tri_2_cr, t_cr = [],[],[]
    rec_cd = cv2.boundingRect(np.float32([interm_pt]))
    tri_1_cr, tri_2_cr, t_cr = Transform_mid(t1, t2, rec_cd, rec_cd_1, rec_cd_2)
    tri_1_crop, tri_2_crop, t_crop = [],[],[]

    rec_coord_2 = cv2.boundingRect(np.float32([tri_2]))
    rec_coord_1 = cv2.boundingRect(np.float32([tri_1]))
    
    tri_2_crop = [(pt[0]-rec_coord_2[0], pt[1]-rec_coord_2[1]) for pt in tri_2]
    tri_1_crop = [(pt[0]-rec_coord_1[0], pt[1]-rec_coord_1[1]) for pt in tri_1]
    t_crop = [(pt[0]-rec_coord[0], pt[1]-rec_coord[1]) for pt in interm_pt]

    width, height = rec_coord[2], rec_coord[3]
    # set the size of the output image
    output_size = (width, height)

    # create a mask with the same shape as the region of interest
    mask_shape = (height, width, 3)
    mask = np.zeros(mask_shape, dtype=np.float32)

    # fill the mask with a solid color inside the triangle
    cv2.fillPoly(mask, [np.int32(t_crop)], (1.0, 1.0, 1.0), 16)

    # extract regions of interest from the input images
    x2, y2, w2, h2 = rec_coord_2
    x1, y1, w1, h1 = rec_coord_1
    #region of interest - roi
    img2_roi = img2[y2:y2+h2, x2:x2+w2]
    img1_roi = img1[y1:y1+h1, x1:x1+w1]


    # compute affine transforms for the two input images
    # apply the transforms to the input images
    
    warp2 = cv2.getAffineTransform(np.float32(tri_2_crop), np.float32(t_crop))
    transform_img2 = cv2.warpAffine(img2_roi, warp2, output_size)
    warp1 = cv2.getAffineTransform(np.float32(tri_1_crop), np.float32(t_crop))

    transform_img1 = cv2.warpAffine(img1_roi, warp1, output_size)
    # Compute the weighted sum of the transformed images
    result_roi = cv2.addWeighted(transform_img1, 1.0 - alpha, transform_img2, alpha, 0.0)

    # Extract the region of interest from the input image
    img_roi = img[rec_coord[1]:rec_coord[1] + rec_coord[3], rec_coord[0]:rec_coord[0] + rec_coord[2]]

    # Compute the weighted sum of the input image and the result ROI
    w2 = (result_roi* mask)
    w1 = img_roi*(1.0- mask)
    img[rec_coord[1]:rec_coord[1] + rec_coord[3], rec_coord[0]:rec_coord[0] + rec_coord[2]] = cv2.add(w1, w2)



In [125]:
def text_file_input(path):
    t1_points = []
    t2_points = []
    num_points = 0
    
    with open(path) as f:
        # Read the first line and get the number of points
        num_points = int(next(f))
        
        # Read the remaining lines and get the points for each triangle
        for line in f:
            points = list(map(int, line.split()))
            t1_points.append((points[0], points[1]))
            t2_points.append((points[2], points[3]))
    
    return t1_points, t2_points, num_points




## **Part A**

In [133]:
# Load images
img2 = cv2.imread('morgan.jpg')
img1 = cv2.imread('ben.jpg')

#Detect face
detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

#Taking user defined input from text file "tile_pts.txt"
tile_pts1, tile_pts2 , no_tile_pts= text_file_input('tile_points_72.txt')

h, w, c = img1.shape
'''0 0 0 0
0 459 0 459
399 459 399 459
399 0 399 0'''
#Adding four corner points to given points read from the file
tile_pts1 += [(0, 0), (0, h-1), (w-1, h-1), (w-1, 0)]
tile_pts2 += [(0, 0), (0, h-1), (w-1, h-1), (w-1, 0)]
# 68 tile points and four corner points
points2 = tile_pts2 
points1 = tile_pts1 

#Initializing an empty list to store triangle indices
ind_pnt_tri = []

# print(type(points1))
triangulation_correspond = delaunay_triangulation(points1, img1)
# print(type(del_tri_1))


for triangle in triangulation_correspond:
    # Extracting the coordinates for each point of the triangle
    point1, point2, point3 = (triangle[0], triangle[1]), (triangle[2], triangle[3]), (triangle[4], triangle[5])

    # Determining the position of each point in the triangulation
    index1, index2, index3 = check_index(point1, points1), check_index(point2, points1), check_index(point3, points1)

    # Creating a tuple of the indices of the points and adding it to ind_pnt_tri
    triangle_indices = (index1, index2, index3)
    ind_pnt_tri.append(triangle_indices)


#Initializing frames and an empty list to store images
num_frames = 30
morph_list_img = []
#Looping through each frame
for f in range(num_frames):  
  # Initializing an empty list to store intermediate frame coordinates  
    intermediate_frame = []
    alph = (1.0 / num_frames )*f
    # Interpolating between corresponding points in both images
    for point1, point2 in zip(points1, points2):
        alp_= 1- alph
        x = int(((alp_) * point1[0]) + (alph * point2[0]))
        y = int(((alp_) * point1[1]) + (alph * point2[1]))
        intermediate_frame.append((x, y))

    # Initializing an empty image
    intermediate_morph = np.zeros_like(img1)
    # Looping through each triangle in the triangulation
    len_tr = len(ind_pnt_tri)
    for j in range(len_tr):
      # Extracting the indices of the points that form the current triangle
        triangle = ind_pnt_tri[j]
        x = triangle[0]
        y = triangle[1]
        z = triangle[2]
        # Extracting the coordinates of the points in the original images
        t2 = [points2[i] for i in (x, y, z)]

        t1 = [points1[i] for i in (x, y, z)]

        # Extracting the coordinates of the points in the intermediate frame
        t = [intermediate_frame[i] for i in (x, y, z)]
        # Applying affine transformation to the triangle and adding it to the morphed image
        Affine_transform(img1, img2, intermediate_morph, t1, t2, t, alph)
    morph_list_img.append(intermediate_morph)
    # cv2_imshow(intermediate_morph)

#making gif of morphed images


# Convert the first input image to RGB format and add it to the GIF
gif = [Image.fromarray(cnvt_colour_bgr2rgb(img1))]

# Convert the morphed images to RGB format and add them to the GIF
for i in range(num_frames):
    gif.append(Image.fromarray(cnvt_colour_bgr2rgb(morph_list_img[i])))

# Convert the second input image to RGB format and add it to the GIF
gif.append(Image.fromarray(cnvt_colour_bgr2rgb(img2)))

# Save the GIF animation
frame = gif[0]
frame.save("PartA.gif", format="GIF", append_images=gif[1:], save_all=True, duration=100, loop=0)



### **Part B**

In [127]:
# Load images
img2 = cv2.imread('morgan.jpg')
img1 = cv2.imread('ben.jpg')

#Detect face
detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Detect facial tile points
tile1 = find_facial_tilepts(img1)
tile2 = find_facial_tilepts(img2)

# 68 tile points and four corner points
points1 = mark_coord(img1) 
points2 = mark_coord(img2) 
# for x,y in points1:
#   print(x,y)
# print(type(points1))
# print(points2)

#Initializing an empty list to store triangle indices
ind_pnt_tri = []

# print(type(points1))
triangulation_correspond = delaunay_triangulation(points1, img1)
# print(type(del_tri_1))


for triangle in triangulation_correspond:
    # Extracting the coordinates for each point of the triangle
    point1, point2, point3 = (triangle[0], triangle[1]), (triangle[2], triangle[3]), (triangle[4], triangle[5])

    # Determining the position of each point in the triangulation
    index1, index2, index3 = check_index(point1, points1), check_index(point2, points1), check_index(point3, points1)

    # Creating a tuple of the indices of the points and adding it to ind_pnt_tri
    triangle_indices = (index1, index2, index3)
    ind_pnt_tri.append(triangle_indices)


#Initializing frames and an empty list to store images
num_frames = 30
morph_list_img = []
#Looping through each frame
for f in range(num_frames):  
  # Initializing an empty list to store intermediate frame coordinates  
    intermediate_frame = []
    alph = (1.0 / num_frames )*f
    # Interpolating between corresponding points in both images
    for point1, point2 in zip(points1, points2):
        alp_= 1- alph
        x = int(((alp_) * point1[0]) + (alph * point2[0]))
        y = int(((alp_) * point1[1]) + (alph * point2[1]))
        intermediate_frame.append((x, y))

    # Initializing an empty image
    intermediate_morph = np.zeros_like(img1)
    # Looping through each triangle in the triangulation
    len_tr = len(ind_pnt_tri)
    for j in range(len_tr):
      # Extracting the indices of the points that form the current triangle
        triangle = ind_pnt_tri[j]
        x = triangle[0]
        y = triangle[1]
        z = triangle[2]
        # Extracting the coordinates of the points in the original images
        t2 = [points2[i] for i in (x, y, z)]

        t1 = [points1[i] for i in (x, y, z)]

        # Extracting the coordinates of the points in the intermediate frame
        t = [intermediate_frame[i] for i in (x, y, z)]
        # Applying affine transformation to the triangle and adding it to the morphed image
        Affine_transform(img1, img2, intermediate_morph, t1, t2, t, alph)
    morph_list_img.append(intermediate_morph)
    # cv2_imshow(intermediate_morph)

#making gif of morphed images


# Convert the first input image to RGB format and add it to the GIF
gif = [Image.fromarray(cnvt_colour_bgr2rgb(img1))]

# Convert the morphed images to RGB format and add them to the GIF
for i in range(num_frames):
    gif.append(Image.fromarray(cnvt_colour_bgr2rgb(morph_list_img[i])))

# Convert the second input image to RGB format and add it to the GIF
gif.append(Image.fromarray(cnvt_colour_bgr2rgb(img2)))

# Save the GIF animation
frame = gif[0]
frame.save("PartB.gif", format="GIF", append_images=gif[1:], save_all=True, duration=100, loop=0)
